In [1]:
import numpy as np
import pandas as pd
from scipy.stats import qmc 
from scipy.interpolate import interp1d 
from tqdm import tqdm
import time

import sys, os, pathlib

NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  
sys.path.insert(0, str(ROOT))
from src.Heston_FFT import heston_price_fft


# params
n_samples = 30_000

S0 = 105.0
q  = 0.0

alpha = 1.5
n = 4096
eta = 0.1


def sample_heston_lhs(
    n_samples: int,
    seed: int = 42,
    moneyness_bounds=(0.6, 1.4),
    T_bounds=(0.09, 1.40),
    r_bounds=(-0.01, 0.10)
) -> pd.DataFrame:
    """
    Generate n_samples plausible Heston parameters via Latin Hypercube Sampling (LHS),
    and also sample option strikes through moneyness m = K / S0.

    Constraints (as implemented by bounds below):
      - v0     in [0.01, 0.09]
      - kappa  in [0.50, 5.00] # not sampled --> m
      - theta  in [0.01, 0.09]
      - sigma_v in [0.10, 1.00]
      - rho    in [-0.95, -0.10]
      - T      in [0.09, 1.40]
      - r      in [-0.01, 0.10]
      - moneyness m in [m_low, m_high]
      - Feller: 2*kappa*theta >= sigma_v^2

    Returns
    -------
    pd.DataFrame
        Columns: ['S0','K','m','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    # bounds
    m_low, m_high = moneyness_bounds
    if not (m_low > 0 and m_high > m_low):
        raise ValueError("moneyness_bounds must satisfy 0 < m_low < m_high.")
    T_low, T_high = T_bounds
    r_low, r_high = r_bounds

    # order:              v0  kappa theta sigma_v rho   m      T      r
    l_bounds = np.array([0.01, 0.5, 0.01, 0.10, -0.95, m_low, T_low, r_low], dtype=float)
    u_bounds = np.array([0.09, 5.0, 0.09, 1.00, -0.10, m_high, T_high, r_high], dtype=float)

    collected = []
    attempt = 0

    # Repeat in batches until we have enough valid samples
    while len(collected) < n_samples and attempt < 20:
        # Oversample to compensate for filtering
        batch_size = int(np.ceil((n_samples - len(collected)) * 2.0))
        sampler = qmc.LatinHypercube(d=8, seed=seed + attempt)
        U = sampler.random(batch_size)
        X = qmc.scale(U, l_bounds, u_bounds)  # shape [batch_size, 8]

        v0, kappa, theta, sigma_v, rho, m, T_samp, r_samp = (
            X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4], X[:, 5], X[:, 6], X[:, 7]
        )

        # Feller condition to ensure variance stays strictly positive
        feller_ok = (2.0 * kappa * theta) >= (sigma_v ** 2)

        X_ok = X[feller_ok]

        # Append valid rows until reaching n_samples
        for row in X_ok:
            collected.append(row)
            if len(collected) >= n_samples:
                break

        attempt += 1

    if len(collected) < n_samples:
        raise RuntimeError(
            "Could not generate enough samples satisfying constraints. "
            "Relax bounds slightly or increase attempts."
        )

    params = np.vstack(collected)[:n_samples]
    df = pd.DataFrame(params, columns=["v0", "kappa", "theta", "sigma_v", "rho", "m", "T", "r"])

 
    df["S0"] = S0
    df["K"]  = (df["m"] * S0).astype(float)
    df["q"]  = q
   
    df = df[["S0","K","m","T","r","q","v0","kappa","theta","sigma_v","rho"]] # KEEP S0, K, q for pricing

    return df

df_first = sample_heston_lhs(n_samples, seed=123)
df_first.head()


,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,rho
0,105.0,145.475663,1.385483,0.968510,0.069419,0.0,0.032475,4.949821,0.067486,0.700567,-0.161174
1,105.0,94.856562,0.903396,0.677513,0.095451,0.0,0.052604,3.644931,0.084572,0.754469,-0.301977
2,105.0,126.278959,1.202657,0.827106,0.040251,0.0,0.054853,4.100707,0.076639,0.767073,-0.211429
3,105.0,135.334600,1.288901,0.796442,0.056577,0.0,0.022490,4.771485,0.070842,0.487195,-0.363820
4,105.0,88.716391,0.844918,1.350815,0.022171,0.0,0.018988,1.816727,0.076024,0.224556,-0.122988


In [2]:
def price_heston_fft_df(
    df: pd.DataFrame,
    option_type: str = "call",
    alpha: float = 1.5,
    N: int = 4096,
    eta: float = 0.05,
) -> pd.DataFrame:
    """
    Adds the 'price' columns via FFT

    Required cols:
    ['S0','K','T','r','q','v0','kappa','theta','sigma_v','rho'].
    """
    required = ["S0","K","T","r","q","v0","kappa","theta","sigma_v","rho"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing cols: {missing}")

    prices = []
    for _, row in df.iterrows():
        try:
            p = heston_price_fft(
                S=float(row["S0"]),
                K=float(row["K"]),
                T=float(row["T"]),
                r=float(row["r"]),
                v0=float(row["v0"]),
                kappa=float(row["kappa"]),
                theta=float(row["theta"]),
                sigma_v=float(row["sigma_v"]),
                rho=float(row["rho"]),
                option_type=option_type,
                q=float(row["q"]),
                alpha=alpha,
                N=N,
                eta=eta,
            )
        except Exception:
            p = np.nan
        prices.append(p)

    out = df.copy()
    out["price"] = prices
    return out


df = price_heston_fft_df(df_first)
df.head()

,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,rho,price
0,105.0,145.475663,1.385483,0.968510,0.069419,0.0,0.032475,4.949821,0.067486,0.700567,-0.161174,1.867288
1,105.0,94.856562,0.903396,0.677513,0.095451,0.0,0.052604,3.644931,0.084572,0.754469,-0.301977,19.053392
2,105.0,126.278959,1.202657,0.827106,0.040251,0.0,0.054853,4.100707,0.076639,0.767073,-0.211429,3.854632
3,105.0,135.334600,1.288901,0.796442,0.056577,0.0,0.022490,4.771485,0.070842,0.487195,-0.363820,1.827369
4,105.0,88.716391,0.844918,1.350815,0.022171,0.0,0.018988,1.816727,0.076024,0.224556,-0.122988,22.434793


In [ ]:
# random check
price_fft_example = heston_price_fft(S=105, K=74.4210, T=0.4142, r=0.0770, v0=0.0352, kappa=1.2193, theta=0.0685, sigma_v=0.3550, rho=-0.6053,
                             option_type="call", q=0.0, alpha=alpha, N=n, eta=eta)

price_fft_example

32.999084677497365

In [3]:
df.drop(columns=['q', 'S0', 'K'], inplace = True)
df

,m,T,r,v0,kappa,theta,sigma_v,rho,price
0,1.385483,0.968510,0.069419,0.032475,4.949821,0.067486,0.700567,-0.161174,1.867288
1,0.903396,0.677513,0.095451,0.052604,3.644931,0.084572,0.754469,-0.301977,19.053392
2,1.202657,0.827106,0.040251,0.054853,4.100707,0.076639,0.767073,-0.211429,3.854632
3,1.288901,0.796442,0.056577,0.022490,4.771485,0.070842,0.487195,-0.363820,1.827369
4,0.844918,1.350815,0.022171,0.018988,1.816727,0.076024,0.224556,-0.122988,22.434793
...,...,...,...,...,...,...,...,...,...
29995,1.030335,0.935719,-0.009445,0.026025,0.512205,0.060798,0.141865,-0.705007,5.273596
29996,0.769683,0.429437,0.081052,0.022708,3.486140,0.055792,0.335177,-0.221563,27.034597
29997,1.190677,1.106063,0.099378,0.015214,4.455063,0.084398,0.701513,-0.161121,8.463715
29998,0.870061,0.124505,0.050292,0.044118,2.373094,0.065976,0.478162,-0.131963,14.337416


In [ ]:
import os
os.makedirs("data", exist_ok=True)
#df.to_csv("data/df_prova_2.csv", index=False)